#### Loading the dataset 

In [1]:
import os
import urllib.request

TITANIC_PATH = os.path.join("datasets", "titanic")
DOWNLOAD_URL = "https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/titanic/"

def fetch_titanic_data(url=DOWNLOAD_URL, path=TITANIC_PATH):
    if not os.path.isdir(path):
        os.makedirs(path)
    for filename in ("train.csv", "test.csv"):
        filepath = os.path.join(path, filename)
        if not os.path.isfile(filepath):
            print("Downloading", filename)
            urllib.request.urlretrieve(url + filename, filepath)

fetch_titanic_data()  

In [2]:
import pandas as pd

def load_titanic_data(filename, titanic_path=TITANIC_PATH):
    csv_path = os.path.join(titanic_path, filename)
    return pd.read_csv(csv_path)

In [3]:
train_data = load_titanic_data("train.csv")
test_data = load_titanic_data("test.csv")

#### Exploring the dataset 

In [4]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
#setting the passenger id as index column
train_data = train_data.set_index("PassengerId")

In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [7]:
train_data.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699113,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526507,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.416700,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
train_data["Survived"].value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [9]:
train_data["Pclass"].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [10]:
train_data["Sex"].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [11]:
train_data["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

#### Preprocessing 

In [12]:
#Building the preprocessing pipeline
#pipeline for numerical attributes

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ("imputer",SimpleImputer(strategy="median")),
    ("scaler",StandardScaler())
])

In [13]:
#building pipeline for categorical attributes
from sklearn.preprocessing import OneHotEncoder
cat_pipeline = Pipeline([
    ("imputer",SimpleImputer(strategy="most_frequent")),
    ("cat_encoder",OneHotEncoder(sparse=False))
])

In [14]:
#Joining the numerical and categorical pipeline
from sklearn.compose import ColumnTransformer

num_attribs = ["Age","SibSp","Parch","Fare"]
cat_attribs = ["Pclass","Sex","Embarked"]

preprocessing_pipeline = ColumnTransformer([
    ("num",num_pipeline,num_attribs),
    ("cat",cat_pipeline,cat_attribs),
])

In [15]:
X_train = preprocessing_pipeline.fit_transform(train_data[num_attribs+cat_attribs])

In [16]:
X_train

array([[-0.56573582,  0.43279337, -0.47367361, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.6638609 ,  0.43279337, -0.47367361, ...,  1.        ,
         0.        ,  0.        ],
       [-0.25833664, -0.4745452 , -0.47367361, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.10463705,  0.43279337,  2.00893337, ...,  0.        ,
         0.        ,  1.        ],
       [-0.25833664, -0.4745452 , -0.47367361, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.20276213, -0.4745452 , -0.47367361, ...,  0.        ,
         1.        ,  0.        ]])

In [17]:
#getting labels
y_train = train_data["Survived"]

In [18]:
X_train.shape

(891, 12)

#### Training the model 

In [19]:
import tensorflow as tf
from tensorflow import keras

In [60]:
model = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(200,activation="relu"),
    keras.layers.Dense(100,activation="relu"),
    keras.layers.Dense(10,activation="softmax")
])

In [61]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer = "sgd",
             metrics=["accuracy"])

In [62]:
history = model.fit(X_train,y_train,epochs=50)

Epoch 1/50
28/28 [==============================] - 0s 1ms/step - loss: 1.9718 - accuracy: 0.4747
Epoch 2/50
28/28 [==============================] - 0s 1ms/step - loss: 1.3869 - accuracy: 0.6251
Epoch 3/50
28/28 [==============================] - 0s 1ms/step - loss: 0.9964 - accuracy: 0.6386
Epoch 4/50
28/28 [==============================] - 0s 1ms/step - loss: 0.7870 - accuracy: 0.6689
Epoch 5/50
28/28 [==============================] - 0s 1ms/step - loss: 0.6699 - accuracy: 0.7295
Epoch 6/50
28/28 [==============================] - 0s 1ms/step - loss: 0.6013 - accuracy: 0.7800
Epoch 7/50
28/28 [==============================] - 0s 2ms/step - loss: 0.5588 - accuracy: 0.8070
Epoch 8/50
28/28 [==============================] - 0s 1ms/step - loss: 0.5290 - accuracy: 0.8103
Epoch 9/50
28/28 [==============================] - 0s 1ms/step - loss: 0.5087 - accuracy: 0.8092
Epoch 10/50
28/28 [==============================] - 0s 1ms/step - loss: 0.4931 - accuracy: 0.8058
Epoch 11/50
28/28 [

In [23]:
#making prediction on test set
X_test = preprocessing_pipeline.transform(test_data[num_attribs+cat_attribs])

In [24]:
PassengerId = test_data[["PassengerId"]]

In [69]:
y_pred = model.predict(X_test)
y_pred.round(2)

array([[0.93, 0.07, 0.  , ..., 0.  , 0.  , 0.  ],
       [0.59, 0.41, 0.  , ..., 0.  , 0.  , 0.  ],
       [0.92, 0.08, 0.  , ..., 0.  , 0.  , 0.  ],
       ...,
       [0.95, 0.05, 0.  , ..., 0.  , 0.  , 0.  ],
       [0.93, 0.07, 0.  , ..., 0.  , 0.  , 0.  ],
       [0.86, 0.13, 0.  , ..., 0.  , 0.  , 0.  ]], dtype=float32)

In [72]:
import numpy as np
y_pred = np.argmax(model.predict(X_test),axis=1)
y_pred

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [95]:
submission = test_data[["PassengerId"]]
submission["Survived"] = y_pred

C:\Users\Intel\AppData\Local\Temp/ipykernel_7108/1538914212.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission["Survived"] = y_pred


In [96]:
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [97]:
submission.to_csv("submission.csv",index=False)